In [1]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from ipywidgets import interactive, fixed  
# ipwidgets makes it easy to make Jupyter notebooks interactive
#  * interactive() for making function inputs interactive
#  * fixed() allows params to be fixed


In [2]:
# Plot the Fourier transform of (mono) signal sampled at sample_rate
def PlotFFT(signal, sample_rate, Nfft = 2048, start = 1024, log_scale = True):
    """ Plot the Fourier transform of (mono) `signal` that is sampled 
    at `sample_rate`.  Uses an `Nfft` point DFT for data starting at  
    sample number `start`.  (Does not return anything, just makes the plot.)
    """
    w = np.hanning(Nfft)
    x_fft = np.fft.fft(w * signal[start: start + Nfft])[:Nfft//2]
    f_fft = np.arange(Nfft//2) * sample_rate/Nfft
    if log_scale:
        plt.xlim(100, 20_000)
        plt.semilogx(f_fft, 10.*np.log10( np.abs( x_fft )**2 *4 * 8/3 /Nfft**2))
    else:
        plt.xlim(0, 20_000)
        plt.plot(f_fft, 10.*np.log10( np.abs( x_fft )**2 *4 * 8/3 /Nfft**2))
    plt.ylim(-80,10)
    plt.xlabel("Frequency in Hz")
    plt.ylabel("dB")
    plt.title("DFT of Signal")
    plt.gca().get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    plt.show()

In [3]:
# Carry out FM synthesis and show the result 
def fm_synth(fc = 2_000, fm = 1_500, m = 5., max_time = 3., rate = 44_100, log_scale = True):
    """Creates an FM modulated signal where the carrier has frequency `fc`,
    the modulating signal has frequency `fm`, and the modulation index is `m`.
    It then plays `max_time` seconds of the signal sampled with sample rate
    `rate` and displays if DFT (computed using `PlotFFT()` defined above.)
    """
    times = np.linspace(0,max_time,rate*max_time+1)
    signal = np.sin(2*np.pi*fc*times + m * np.sin(2*np.pi*fm*times))
    display(Audio(data=signal, rate=rate, autoplay = True))
    PlotFFT(signal, rate, log_scale=log_scale)
    return signal

# interactively run the function `fm_synth()`
w = interactive( fm_synth
               , fc=(500, 5_000)
               , fm=(500, 5_000)
               , m = (0., 30.)
               , max_time = fixed(3)
               , rate = fixed(44_100)
               , log_scale = True
               )
display(w)

interactive(children=(IntSlider(value=2000, description='fc', max=5000, min=500), IntSlider(value=1500, descri…